In [1]:
!pip install -q llama-index==0.9.14.post3 deeplake==3.8.8 openai==1.3.8 cohere==4.37

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.5/943.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 kB 36.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.5/221.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.

In [13]:
import os

os.environ['OPENAI_API_KEY'] = '<YOUR_OPENAI_API_KEY>'
os.environ['ACTIVELOOP_TOKEN'] = '<YOUR_ACTIVELOOP_API_KEY>'

In [8]:
%mkdir -p  './data/1k/'
!wget 'https://github.com/idontcalculate/data-repo/blob/main/machine_to_end_war.txt' -O './data/1k/tesla.txt'
!wget 'https://github.com/idontcalculate/data-repo/blob/main/prodigal_chapter10.txt' -O './data/1k/web.txt'

--2023-12-27 12:32:45--  https://github.com/idontcalculate/data-repo/blob/main/machine_to_end_war.txt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18608 (18K) [text/plain]
Saving to: ‘./data/1k/tesla.txt’

./data/1k/tesla.txt 100%[===================>]  18.17K  --.-KB/s    in 0.02s   

2023-12-27 12:32:45 (728 KB/s) - ‘./data/1k/tesla.txt’ saved [18608/18608]

--2023-12-27 12:32:45--  https://github.com/idontcalculate/data-repo/blob/main/prodigal_chapter10.txt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28208 (28K) [text/plain]
Saving to: ‘./data/1k/web.txt’

./data/1k/web.txt   100%[===================>]  27.55K  --.-KB/s    in 0.02s   

2023-12-27 12:32:45 (1.09 MB/s) - ‘./data/1k/web.txt’ saved [28208/28208]



In [9]:
from llama_index import SimpleDirectoryReader

tesla_docs = SimpleDirectoryReader( input_files=["/content/data/1k/tesla.txt"] ).load_data()

/usr/local/lib/python3.10/dist-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.13) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [10]:
from llama_index.vector_stores import DeepLakeVectorStore

my_activeloop_org_id = "doukansurel"
my_activeloop_dataset_name = "LlamaIndex_tesla_predictions"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

In [11]:
# Create an index over the documnts

vector_store = DeepLakeVectorStore(dataset_path=dataset_path, overwrite=False)

Your Deep Lake dataset has been successfully created!


In [14]:
from llama_index.storage.storage_context import StorageContext
from llama_index import VectorStoreIndex

storage_context  = StorageContext.from_defaults(vector_store=vector_store)
tesla_index = VectorStoreIndex.from_documents(tesla_docs,storage_context=storage_context)

Uploading data to deeplake dataset.


100%|██████████| 5/5 [00:00<00:00,  7.32it/s]
|

Dataset(path='hub://doukansurel/LlamaIndex_tesla_predictions', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (5, 1)      str     None   
 metadata     json      (5, 1)      str     None   
 embedding  embedding  (5, 1536)  float32   None   
    id        text      (5, 1)      str     None   


In [15]:
#Store Index Local
webtext_docs = SimpleDirectoryReader(input_files=["/content/data/1k/web.txt"]).load_data()

In [16]:
try:
  # Try to load the index if it is already calculated
  storage_context = StorageContext.from_defaults( persist_dir="/content/storage/webtext" )
  webtext_index = load_index_from_storage(storage_context)
  print("Loaded the pre-computed index.")
except:
  # Otherwise, generate the indexes
  webtext_index = VectorStoreIndex.from_documents(webtext_docs)
  webtext_index.storage_context.persist(persist_dir="/content/storage/webtext")
  print("Generated the index.")

Generated the index.


Step 2: Query Engine

In [17]:
tesla_engine = tesla_index.as_query_engine(similarity_top_k=3)
webtext_engine = webtext_index.as_query_engine(similarity_top_k=3)

In [18]:
from llama_index.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=tesla_engine,
        metadata = ToolMetadata(
            name="tesla_1k",
            description=(
                 "Provides information about Tesla's statements that refers to future times and predictions. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),

    ),

  QueryEngineTool(
      query_engine=webtext_engine,
      metadata=ToolMetadata(
          name="webtext_1k",
          description=(
                "Provides information about tesla's life and biographical data. "
                "Use a detailed plain text question as input to the tool."
            ),
      ),
  ),

]

İşte mevcut sistemimizin açık bir şematik temsili. Sorgu motoru üstte tasvir edilmiştir ve bu da onun her şeyi düzenleyen birincil araç rolünü belirtir. Veri kaynakları ile nihai cevabı formüle etme süreci arasında merkezi bir konuma sahiptir. Önerilen sorular ile bunların yanıtları arasında bir köprü görevi görür. LlamaIndex ile temel RAG mekanizmasını kurduktan sonraki adım bir aracının entegrasyonudur. Bu ekleme, geri alma sisteminin kolay test edilmesini sağlar. Temel işlevsellik test edilip doğrulandıktan sonra sistem tasarımı iyileştirmeleri ve özellik geliştirmeleri ekleyebiliriz.

Step 3: The Agent

In [19]:
from llama_index.agent import OpenAIAgent
agent =OpenAIAgent.from_tools(query_engine_tools,verbose=True)


Artık temsilcimize sahip olduğumuza göre, aracının girdileri (sorular veya istemler gibi) alabileceği, bunları işleyebileceği ve yanıtları geri döndürebileceği etkileşimli bir sohbet arayüzü (REPL, Okuma-Değerlendirme-Yazdırma Döngüsü) çalıştırabiliriz; bu da onu konuşmaya dayalı bir aracı yapabilir bir diyalog veya sohbet oturumunu yönetme.

In [21]:
agent.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.

Human: What influenced Nikola Tesla to become an inventor?
STARTING TURN 1
---------------

=== Calling Function ===
Calling function: webtext_1k with args: {
"input": "What influenced Nikola Tesla to become an inventor?"
}
Got output: Nikola Tesla was influenced to become an inventor by his observations and studies of mechanical vibrations. He noticed that objects responded differently to vibrations and believed that resonance conditions could produce effects of tremendous magnitude on physical objects. This led him to explore the field of mechanical vibrations and develop inventions based on this principle.

STARTING TURN 2
---------------

Assistant: Nikola Tesla was influenced to become an inventor by his observations and studies of mechanical vibrations. He noticed that objects responded differently to vibrations and believed that resonance conditions could produce effects of tremendous magnitude on physical objects. This led hi

In [22]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

from llama_index.tools import FunctionTool

multiply_tool = FunctionTool.from_defaults(fn=multiply, name="multiply")
add_tool = FunctionTool.from_defaults(fn=add, name="add")

all_tools = [multiply_tool, add_tool]

Yukarıdaki kod 'ekleme' ve 'çarpma' başlıklı iki işlev oluşturur. Bu kurulumda, giriş argümanları için veri türlerini (a:int, b:int), işlevin dönüş türünü (->int) ve üçlü tırnak işaretleri içinde sağlanan işlevin amacının kısa bir açıklamasını belirtmek çok önemlidir. işlev adının altında. Bu ayrıntılar, daha sonra aracı tarafından kullanılabilecek işlevin bir açıklamasını oluşturmak için FunctionTool sınıfının .from_defaults() yöntemi tarafından kullanılacaktır. Son değişken mevcut tüm araçların bir listesini içerir. Bu araçlar, VectorStoreIndex'i birden fazla olası araca bağlayan bir sarmalayıcı sınıf olan ObjectIndex'i oluşturmak için kullanılabilir. Başlangıçta, araç uygulamalarını düğümlere dönüştürmek ve ardından her şeyi birbirine bağlamak için SimpleToolNodeMapping aracını kullanmak gerekir.

In [23]:
from llama_index import VectorStoreIndex
from llama_index.objects import ObjectIndex, SimpleToolNodeMapping

tool_mapping = SimpleToolNodeMapping.from_objects(all_tools)
onj_index = ObjectIndex.from_objects(
    all_tools,
    tool_mapping,
    VectorStoreIndex,
)

Bu uygulamaya herhangi bir veri kaynağı dahil etmediğimizi unutmayın. Büyük Dil Modellerinin yeteneklerini ek araçlarla geliştirmeyi hedeflediğimiz için bu yaklaşım kasıtlıdır. Bir sonraki kod bloğunda tanımlı nesne indeksini alıcı olarak kullandığımızı göreceksiniz! Bu, LlamaIndex çerçevesinde özel işlevlerin ek veri kaynakları olarak ele alındığı anlamına gelir. Bu nedenle, aracı nesnesini FnRetrieverOpenAIAgent sınıfını kullanarak tanımlıyoruz.

In [24]:
from llama_index.agent import FnRetrieverOpenAIAgent

agent = FnRetrieverOpenAIAgent.from_retriever(
    onj_index.as_retriever(),verbose = True
)

In [25]:
agent.chat("What's 12 multiplied by 22? Make sure to use Tools")

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: multiply with args: {
  "a": 12,
  "b": 22
}
Got output: 264

STARTING TURN 2
---------------



AgentChatResponse(response='12 multiplied by 22 is 264.', sources=[ToolOutput(content='264', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 12, 'b': 22}}, raw_output=264)], source_nodes=[])

Önceki örnekte, istemde aracının araçları kullanması gerektiğini belirtmiştik. Ek olarak, aracıyı belirli araçları kullanmaya açıkça yönlendirmek için tool_choice argümanını kullanmak veya aracının karar vermesine izin vermek için auto anahtar sözcüğünü kullanmak mümkündür.

In [26]:
response = agent.chat( "What is 5 + 2?", tool_choice="add" )

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: add with args: {
  "a": 5,
  "b": 2
}
Got output: 7

STARTING TURN 2
---------------

